# 自然语言识别处理单元
## [拂晓工作室](https://github.com/Errrneist/Alchemist)
* 此程序可用于识别文字中的语言以及深度分析新闻概况

# 参考资料
* [1] [Apple - SFrame实例](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html#turicreate.SFrame)
* [2] [Apple - Turicreate中关于SFrame.apply()的文档](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.apply.html)
* [0] [Apple - Turicreate使用TrimRareWords来快速定义高频词](https://apple.github.io/turicreate/docs/api/generated/turicreate.text_analytics.trim_rare_words.html?highlight=remove%20punctuation)
* [0] [Apple - 用Turicreate生成Word Count Vector](https://apple.github.io/turicreate/docs/api/generated/turicreate.text_analytics.count_words.html)
* [0] [Apple - 关于WordProcessing以及使用dict_trim_by_values(n)删除频率以下的词以及TF-IDF的描述](https://turi.com/learn/userguide/text/analysis.html)
* [3] [Stackoverflow - Python从字符串中删除子字符串](https://stackoverflow.com/questions/31273642/better-way-to-remove-multiple-words-from-a-string)
* [4] [Stackoverflow - 关于SFrame.apply()和Lambda x的实例](https://stackoverflow.com/questions/33028423/how-can-i-use-apply-with-a-function-that-takes-multiple-inputs)
* [0] [Stackexchange - Merge two list and discarding duplicates](https://codereview.stackexchange.com/questions/108171/merge-two-list-and-discarding-duplicates)
* [5] [楼宇 - 用Python分析《红楼梦》](https://zhuanlan.zhihu.com/p/29209681)
* [0] [zhon - hanzi文档](http://zhon.readthedocs.io/en/latest/)
* [6] [CSDN - 使用pyltp包进行中文分词实例](https://blog.csdn.net/sinat_26917383/article/details/77067515)
* [7] [CSDN - NLP+语义分析: 角色标注、篇章分析](https://blog.csdn.net/sinat_26917383/article/details/55683599)
* [8] [CSDN - NLP情感分析简介](https://blog.csdn.net/android_ruben/article/details/78174172)
* [9] [CSDN - 基于机器学习的NLP情感分析（二）—- 分类问题](https://blog.csdn.net/stary_yan/article/details/75330729)
* [0] [CSDN - 关于使用zhon.hanzi移除标点符号的解决方法](https://www.cnblogs.com/arkenstone/p/6092255.html)
* [0] [CSDN - Python中正则表达式sub函数用法总结](https://blog.csdn.net/hzliyaya/article/details/52495150)
* [0] [CSDN - Python中另一种移除标点符号的办法](https://blog.csdn.net/mach_learn/article/details/41744487)
* [0] [CSDN - Python字典从大到小排序](https://blog.csdn.net/yangnianjinxin/article/details/79284318)
* [0] [CSDN - 中文依存句法简介](https://blog.csdn.net/sinat_33741547/article/details/79258045)
* [10] [PYLTP - pyltp技术文档](http://pyltp.readthedocs.io/zh_CN/latest/api.html#id15)
* [11] [PYLTP - pyltp介绍文档一](https://www.ltp-cloud.com/intro/#introduction )
* [12] [PYLTP - pyltp介绍文档二](http://ltp.readthedocs.io/zh_CN/latest/appendix.html#id5)
* [13] [PYLTP - pyltp深度训练模型](https://pan.baidu.com/share/link?shareid=1988562907&uk=2738088569#list/path=/)


# [A] 导入库模块

In [1]:
# 导入基本库
import re
import os
import time
import random
import datetime

# 网络获取相关包
import pymysql
import urllib
from bs4 import BeautifulSoup

# 机器学习与大数据框架
import turicreate as tc
import csv
from zhon.hanzi import punctuation

# 自然语言识别框架
import pyltp
from pyltp import SentenceSplitter
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import NamedEntityRecognizer
from pyltp import Parser
from pyltp import SementicRoleLabeller
LTP_DATA_DIR = '../../../LTP_data_v3.4.0/'  # ltp模型目录的路径
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')  # 命名实体识别模型路径，模型名称为`pos.model`
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`
srl_model_path = os.path.join(LTP_DATA_DIR, 'srl')  # 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。


# [B] 载入数据模块

In [ ]:
# 用于从上次停止工作后恢复数据
news_path = '../DataSets/Eastmoney/News_NLP/China/'
news_source = 'NLPCHINA20180702-1906'
# news_source = 'NLPCHINA20180702-1906.csv'
news = tc.SFrame(news_path + news_source)

In [2]:
# 载入纯净数据
news_path = '../DataSets/Eastmoney/News/China/'
news_source = 'CHINA20180702-1906.csv'
news = tc.SFrame(news_path + news_source)

# 预览
# news

Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180702-1906.csv

Parsing completed. Parsed 100 lines in 0.053299 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,int,list,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180702-1906.csv

Parsing completed. Parsed 509 lines in 0.042847 secs.

# [C] 检查数据纯净度模块
* 测试用，检查清洗数据的效果

In [ ]:
# 随机人工检查数据纯净度

def showContents(number_to_show, total_amount):
    counter = 0
    while counter < number_to_show:
        print(news['contents'][int(random.uniform(0, total_amount - 1))])
        print('')
        counter += 1
    
# 预览
# showContents(3, len(news['contents']))

# [D] 自然语言识别模块

## [1] 分句模块

### 分句函数
* 将句子分开并创建名为'content_split_sents'的列

In [3]:
# 分句函数
def splitSents(contents):
    return SentenceSplitter.split(contents)

# 执行分句函数
news['content_split_sents'] = news['contents'].apply(splitSents)

# 预览
# news

##  [2] 移除标点符号模块

In [4]:
# 定义移除标点符号函数
def removePunctuation(contents):
    return re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "", contents)

# 运行移除函数并创建一个名为'contents_nopunc'的列
news['contents_nopunc'] = news['contents'].apply(removePunctuation)

<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:3: DeprecationWarning: invalid escape sequence \s
<ipython-input-4-708563a9aa48>:3: DeprecationWarning: invalid escape sequence \s
  return re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "", contents)
<unknown>:2: DeprecationWarning: invalid escape sequence \s


## [3] 分词函数模块
* 将词分开 并创建名为'contents_split_words'的列

In [5]:
# 分词函数
def splitWords(contents):
    segmentor = Segmentor()  # 初始化实例
    segmentor.load(cws_model_path)  # 加载模型
    words = segmentor.segment(contents)  # 分词
    segmentor.release()  # 释放模型
    return words

# 执行分词函数
news['content_split_words'] = news['contents_nopunc'].apply(splitWords)

# 预览
# news

## [4] 词性标注模块

In [6]:
def tagWords(contents_split):
    postagger = Postagger() # 初始化实例
    postagger.load(pos_model_path)  # 加载模型

    words = contents_split  # 分词结果
    postags = postagger.postag(words)  # 词性标注

    postagger.release()  # 释放模型
    return postags

# 执行标注函数
news['content_tag_words'] = news['content_split_words'].apply(tagWords)

# 预览
# news

## [5] 命名实体识别模块

In [7]:
# 讲究实体识别
def recognizeWords(words):
    # postags = news['content_split_words' == words]['content_tag_words']
    recognizer = NamedEntityRecognizer() # 初始化实例
    recognizer.load(ner_model_path)  # 加载模型
    netags = recognizer.recognize(words, news['content_split_words' == words]['content_tag_words'])  # 命名实体识别
    recognizer.release()  # 释放模型
    return netags

# 执行实体识别
news['recognized_words'] = news['content_split_words'].apply(recognizeWords)

## [6] 生成Word Count Vector模块

In [8]:
# 生成Word Count Vector
news['word_count'] = tc.text_analytics.count_words(news['content_split_words'])

# 整理Word Count Vector
def sortWordCount(dictionary):
    return sorted(dictionary.items(),key = lambda d:d[1],reverse=True)

# 执行整理Word Count Vector
news['sorted_word_count'] = news['word_count'].apply(sortWordCount)

## [7] 计算TF-IDF模块

In [9]:
# 主要生成TF-IDF模块

# 生成空格函数
def separateSpace(wordlist):
    temp_str = ''
    for word in wordlist:
        temp_str = temp_str + word + ' '
    return temp_str

# 生成空格组成的str 
news['contents_space'] = news['content_split_words'].apply(separateSpace)

# 通过由空格组成的str计算TF-IDF
news['tf-idf'] = tc.text_analytics.tf_idf(news['contents_space'])

# 整理TF-IDF
def sortTFIDF(dictionary):
    return sorted(dictionary.items(),key = lambda d:d[1],reverse=True)

# 执行整理TF-IDF
news['sorted_tf-idf'] = news['tf-idf'].apply(sortTFIDF)

In [15]:
# TF-IDF关键字排名模块
def rankKeyTFIDF(tfidf_list):
    origional_list = tfidf_list[0:10]
    keyWord_list = []
    for item in origional_list:
        keyWord_list.append(item[0])
    return keyWord_list

news['ranked_tf-idf_keywords'] = news['sorted_tf-idf'].apply(rankKeyTFIDF)

## [8] 依存句法分析模块（实验中）
* 似乎还不支持保存为pyltp.ParsedWords格式……

In [ ]:
# 依存句法分析函数
def parseWords(words):
    postags = news['content_split_words' == words]['content_tag_words']
    parser = Parser() # 初始化实例
    parser.load(par_model_path)  # 加载模型
    arcs = parser.parse(words, postags)  # 句法分析
    parser.release()  # 释放模型
    return arcs

# 执行依存句法分析
news['parsed_words'] = news['content_split_words'].apply(parseWords)

## [9] 语义角色分析模块（实验中）
* 似乎还不支持分析或保存为pyltp.ParsedWords格式……

In [ ]:
# 语意角色分析函数
def labelWords(words):
    postags = news['content_split_words' == words]['content_tag_words']
    netags = news['content_split_words' == words]['recognized_words']
    arcs = news['content_split_words' == words]['parsed_words']
    
    labeller = SementicRoleLabeller() # 初始化实例
    labeller.load(srl_model_path)  # 加载模型
    roles = labeller.label(words, postags, netags, arcs)
    labeller.release()  # 释放模型
    return roles

# 执行语意角色分析
news['labeled_words'] = news['content_split_words'].apply(labelWords)

## [10] 创建关键字模块

In [21]:
# Merge two list and discarding duplicates [Stackexchange]
def merge_no_duplicates(iterable_1, iterable_2):
    myset = set(iterable_1).union(set(iterable_2))
    return sorted(list(myset))

# 将TFIDF筛选出的和从网站本身下载的超链关键字放进一个关键字SArray
def makeKeyWords(related):
    ranked_tfidf = news['related' == related]['ranked_tf-idf_keywords']
    return merge_no_duplicates(related, ranked_tfidf)

# 执行创建关键字
news['keywords'] = news['related'].apply(makeKeyWords)

## [11] 预览SFrame模块

In [19]:
# news

## [10] 随机检查成果模块

In [18]:
def printRank(num):
    print('目前显示第' + str(num) + '条新闻：')
    print('----------------------------------------------------------------')
    print(news[num]['contents'])
    print('----------------------------------------------------------------')
    print(news[num]['ranked_tf-idf_keywords'])
    print('----------------------------------------------------------------')
    print(news[num]['related'])
    
printRank(random.randint(0,len(news['title'])))

目前显示第45条新闻：
----------------------------------------------------------------
据工信部网站消息，日前工信部、应急管理部、财政部、科技部联合发布了《关于加快安全产业发展的指导意见》。意见指出，安全产业是为安全生产、防灾减灾、应急救援等安全保障活动提供专用技术、产品和服务的产业，是国家重点支持的战略产业。发展安全产业对于落实安全发展理念、提升全社会安全保障能力和本质安全水平、推动经济高质量发展、培育新经济增长点具有重要意义。为落实《中共中央国务院关于推进安全生产领域改革发展的意见》(中发〔2016〕32号)，现就安全产业发展提出如下意见。一、总体要求(一)指导思想全面贯彻党的十九大精神，以习近平新时代中国特色社会主义思想为指导，牢固树立安全发展理念，弘扬生命至上、安全第一的思想，聚焦风险隐患源头治理，以坚决遏制重特大安全生产事故为目标，以提升安全保障能力为重点，以示范工程为依托，着力推广先进安全技术、产品和服务，提升各行业领域的本质安全水平；以企业为主体，市场为导向，强化政府引导，着力推动安全产业创新发展、集聚发展，积极培育新的经济增长点。(二)基本原则创新驱动，优化供给。加快关键、亟需新技术新产品研发，提高安全产品供给质量，不断缩小与国际先进水平差距；加快推动商业模式创新，深化产融合作，积极培育安全服务新业态。突出重点，集聚发展。聚焦安全生产事故高发、频发的重点行业领域，优先发展可有效防范事故、具有重大推广应用价值的专用技术与产品；提高产业集中度，完善产业链，促进产业发展规模化、专业化、集聚集约化。需求牵引，示范带动。提升安全标准，强化安全监管，激发市场需求，推广先进可靠的安全产品和服务；面向重点行业领域，坚持问题导向，实施安全产品试点示范应用工程，引导社会资本投入，有力拉动安全产业发展。规范引导，有序推进。充分发挥市场在资源配置中的决定性作用，调动市场主体发展安全产业的积极性；加强行业自律，规范市场秩序，营造有利于安全产业健康发展的市场环境。(三)工作目标到2020年，安全产业体系基本建立，产业销售收入超过万亿元。先进安全产品有效供给能力显著提高，在重点行业领域实现示范应用。创新能力明显提高。突破一批保障生产安全、城市公共安全的关键核心技术，研发一批具有国际先进水平的安全与应急产品，推广应用

## [11] 保存数据模块

In [17]:
# 保存数据
filepath = '../DataSets/Eastmoney/News_NLP/China/'
news_source = 'NLPCHINA20180702-1906'
news.save(filepath + news_source)
# news.save(filepath + 'NLP' + news_source, format='csv')

# 打印时间戳
print('程序运行时间戳：20' 
      + str(datetime.datetime.now().strftime("%y")) + '年'
      + str(datetime.datetime.now().strftime("%m")) + '月' 
      + str(datetime.datetime.now().strftime("%d")) + '日' 
      + str(datetime.datetime.now().strftime("%H")) + '时' 
      + str(datetime.datetime.now().strftime("%M")) + '分' 
      + str(datetime.datetime.now().strftime("%S")) + '秒')

# 打印数据路径
print('\n成功保存自然语言识别数据文件！数据路径：')
print(filepath + news_source)

程序运行时间戳：2018年07月04日20时46分27秒

成功保存自然语言识别数据文件！数据路径：
../DataSets/Eastmoney/News_NLP/China/NLPCHINA20180702-1906
